# Part II — Core Django  
## 12. Authentication and Authorization (Django Auth System)

This chapter upgrades your app from “anyone can create/edit content” to an
industry-standard, secure workflow:

- users can log in and log out
- create/edit actions require authentication
- edits are limited to authorized users (ownership + permissions)
- you understand where login state lives (sessions)
- you can build password reset flows (dev-friendly, production-ready structure)
- you can test auth-protected views properly

We will use Django’s built-in auth system first (industry standard), then layer your
app’s authorization rules on top.

---

## 12.0 Learning Outcomes

By the end of this chapter you should be able to:

1. Explain **authentication** vs **authorization** clearly.
2. Explain how Django auth works internally at a practical level:
   - sessions + cookies
   - `AuthenticationMiddleware` + `request.user`
3. Use Django’s built-in auth views (login/logout/password reset).
4. Protect views with:
   - `@login_required`
   - `@permission_required` (where appropriate)
5. Implement **object-level authorization** (ownership rules):
   - only authors (or staff) can edit/delete an article
6. Add an `author` field to your `Article` model correctly (with migrations).
7. Use `commit=False` + `save_m2m()` correctly in create/edit.
8. Configure redirect behavior (`next` parameter, login redirect).
9. Assign permissions and groups in Django admin.
10. Write tests verifying:
   - anonymous users are redirected to login
   - logged-in users can create/edit
   - unauthorized users are denied

---

## 12.1 Authentication vs Authorization (Do Not Mix These)

### Authentication: “Who are you?”
Examples:
- user enters username/password → system verifies → session created
- request has `request.user` = that user

### Authorization: “Are you allowed to do this?”
Examples:
- only logged-in users can create an article
- only the author can edit their article
- only users with `articles.delete_article` can delete articles

A user can be authenticated but still not authorized.

---

## 12.2 How Django Auth Works (Practical Architecture)

### 12.2.1 Middleware is the key
Two middleware classes make “logged-in state” work:

- `SessionMiddleware`:
  - loads session data using the session cookie (usually `sessionid`)
- `AuthenticationMiddleware`:
  - uses session data to populate `request.user`

If these middleware are missing or out of order, `request.user` won’t behave
correctly.

In `config/settings.py`, ensure you have:

```python
MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    # (your timing middleware if you kept it)
    "django.contrib.sessions.middleware.SessionMiddleware",
    "django.middleware.common.CommonMiddleware",
    "django.middleware.csrf.CsrfViewMiddleware",
    "django.contrib.auth.middleware.AuthenticationMiddleware",
    "django.contrib.messages.middleware.MessageMiddleware",
    "django.middleware.clickjacking.XFrameOptionsMiddleware",
]
```

### 12.2.2 Where login state lives
- Browser stores a cookie (session id).
- Server stores session data (DB by default in Django, unless configured otherwise).

So “logged in” typically means:
- session cookie exists
- session maps to user id
- `request.user` is set on each request

---

## 12.3 Add Auth URLs (Login/Logout/Password Reset) the Standard Way

Django ships built-in auth views and URL patterns. The industry-standard approach
is to include them.

Edit `config/urls.py`:

```python
from django.contrib import admin
from django.urls import include, path

urlpatterns = [
    path("admin/", admin.site.urls),
    path("", include("pages.urls")),
    path("articles/", include("articles.urls")),
    path("accounts/", include("django.contrib.auth.urls")),
]
```

### What routes you just added
Including `django.contrib.auth.urls` gives you named routes like:

- `/accounts/login/` (name: `login`)
- `/accounts/logout/` (name: `logout`)
- `/accounts/password_change/`
- `/accounts/password_reset/`
- etc.

You can see them in Django docs, but you don’t need to memorize all—just remember:
**include auth URLs once**, then override templates as needed.

---

## 12.4 Create Login/Logout Templates (Django Looks for Specific Names)

Django’s built-in auth views expect templates at specific paths. The most important
one is:

- `registration/login.html`

Create:

```text
templates/
  registration/
    login.html
    logged_out.html
```

### 12.4.1 `templates/registration/login.html`
Create `templates/registration/login.html`:

```django
{% extends "base.html" %}

{% block title %}Login{% endblock %}

{% block content %}
  <h1>Login</h1>

  {% if form.errors %}
    <p class="error">Your username and password didn’t match. Try again.</p>
  {% endif %}

  {% if next %}
    <p>You are being redirected after login.</p>
  {% endif %}

  <form method="post" action="{% url 'login' %}">
    {% csrf_token %}

    <div class="field">
      <label for="{{ form.username.id_for_label }}">Username</label>
      {{ form.username }}
      {% for err in form.username.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <div class="field">
      <label for="{{ form.password.id_for_label }}">Password</label>
      {{ form.password }}
      {% for err in form.password.errors %}<div class="error">{{ err }}</div>{% endfor %}
    </div>

    <input type="hidden" name="next" value="{{ next }}" />

    <button type="submit">Log in</button>
  </form>

  <p>
    <a href="{% url 'password_reset' %}">Forgot password?</a>
  </p>
{% endblock %}
```

#### Why `next` exists (important)
When a user tries to access a protected page, Django redirects them to login with a
query string like:

```text
/accounts/login/?next=/articles/new/
```

That `next` tells Django where to send the user after successful login.

We include:

```django
<input type="hidden" name="next" value="{{ next }}" />
```

so POST preserves it.

### 12.4.2 `templates/registration/logged_out.html`
Create `templates/registration/logged_out.html`:

```django
{% extends "base.html" %}

{% block title %}Logged out{% endblock %}

{% block content %}
  <h1>Logged out</h1>
  <p>You have been logged out.</p>
  <p><a href="{% url 'login' %}">Log in again</a></p>
{% endblock %}
```

---

## 12.5 Update Navigation: Show Login/Logout Based on `user.is_authenticated`

Edit `templates/partials/_nav.html` to include:

```django
<nav class="nav">
  <a href="{% url 'pages:home' %}">Home</a>
  <a href="{% url 'articles:list_html' %}">Articles</a>
  <a href="{% url 'articles:create' %}">New Article</a>

  <span style="margin-left:auto;"></span>

  {% if user.is_authenticated %}
    <span>Hi, {{ user.username }}</span>
    <a href="{% url 'logout' %}">Logout</a>
  {% else %}
    <a href="{% url 'login' %}">Login</a>
  {% endif %}
</nav>
```

### Why `user` is available in templates
Because you have the auth context processor enabled:

```python
"django.contrib.auth.context_processors.auth"
```

So `user` is injected into template context automatically.

---

## 12.6 Protect Views: Require Login for Create/Edit (Baseline Authorization)

Right now anyone can create/edit articles. In industry, create/edit should almost
always require login.

### 12.6.1 Protect create and edit with `@login_required`
In `articles/views.py`, add:

```python
from django.contrib.auth.decorators import login_required
```

Then decorate:

```python
from django.contrib.auth.decorators import login_required
from django.views.decorators.http import require_http_methods

@login_required
@require_http_methods(["GET", "POST"])
def article_create(request):
    ...
```

And:

```python
@login_required
@require_http_methods(["GET", "POST"])
def article_edit(request, slug: str):
    ...
```

#### What `@login_required` actually does
If the user is not authenticated:
- Django returns a 302 redirect to the login URL
- includes `?next=...` so user returns after logging in

---

## 12.7 Configure Login Redirect Behavior (Optional but Common)

Django uses `settings.LOGIN_URL` to know where to send users.

Since you included auth URLs at `/accounts/`, the login URL is `/accounts/login/`.
Django usually finds it, but it’s best to be explicit in professional projects.

In `config/settings.py` add:

```python
LOGIN_URL = "/accounts/login/"
LOGIN_REDIRECT_URL = "/"
LOGOUT_REDIRECT_URL = "/"
```

### Why these are settings (not hardcoded in views)
- consistent behavior across the project
- third-party auth flows rely on these defaults
- reduces duplicated redirect logic

---

## 12.8 Object-Level Authorization (Ownership): Add `author` to Article

Authentication alone is not enough. You need rules like:

- Only the author can edit/delete their own article
- Staff/admin can edit any article

This requires an ownership field.

### 12.8.1 Add `author` field to `Article` model

Edit `articles/models.py`:

```python
from django.conf import settings
from django.db import models
from django.utils import timezone

class Article(models.Model):
    ...
    author = models.ForeignKey(
        settings.AUTH_USER_MODEL,
        on_delete=models.PROTECT,
        related_name="articles",
        null=True,
        blank=True,
    )
    ...
```

#### Why `settings.AUTH_USER_MODEL`
It supports:
- default User model now
- custom user model later without rewriting relations

#### Why `on_delete=PROTECT`
For authored content, you usually don’t want deleting a user to delete all content
accidentally. `PROTECT` prevents deletion if articles exist.

#### Why `null=True` initially (migration safety)
Because you already have existing Article rows in your DB.
If you add a non-null FK, Django will ask for a default to fill existing rows.

**Production-safe migration pattern** is typically:
1. Add field as nullable.
2. Backfill values (data migration).
3. Make field non-null.

We’ll follow the correct pattern.

### 12.8.2 Make migrations and migrate
Run:

```bash
python manage.py makemigrations
python manage.py migrate
```

---

## 12.9 Backfill Existing Articles’ Authors (Production-Style Data Migration)

### 12.9.1 Why this matters
If existing articles have `author = NULL`:
- your “only author can edit” rule becomes ambiguous
- you’ll have to handle “orphaned content”

We’ll set a default author for existing rows.

### Option A (recommended for learning): data migration that sets author to first superuser
Create a migration file manually.

Run:

```bash
python manage.py makemigrations --empty articles --name backfill_article_author
```

Edit the created migration in `articles/migrations/XXXX_backfill_article_author.py`
to:

```python
from django.conf import settings
from django.db import migrations


def set_author(apps, schema_editor):
    Article = apps.get_model("articles", "Article")
    User = apps.get_model(*settings.AUTH_USER_MODEL.split("."))

    admin = User.objects.filter(is_superuser=True).order_by("id").first()
    if not admin:
        return

    Article.objects.filter(author__isnull=True).update(author=admin)


class Migration(migrations.Migration):
    dependencies = [
        ("articles", "0002_add_author"),  # adjust to your actual dependency
    ]

    operations = [
        migrations.RunPython(set_author, migrations.RunPython.noop),
    ]
```

Then run:

```bash
python manage.py migrate
```

#### Explanation (what’s special here)
- We use `apps.get_model(...)` instead of importing models directly:
  - migrations must work with historical versions of models
- We find a superuser and assign it as author for existing articles.

### Option B (fast but not production-safe): wipe local DB
If this is purely a learning DB and you don’t care about data:
- delete sqlite DB
- run migrations
- reseed data

This is common in tutorials, but the migration approach above is the professional
pattern you should learn.

---

## 12.10 Make `author` Required (Finish the Safe Migration)

Once backfilled, make field non-nullable.

Edit `articles/models.py`:

```python
author = models.ForeignKey(
    settings.AUTH_USER_MODEL,
    on_delete=models.PROTECT,
    related_name="articles",
)
```

Now:

```bash
python manage.py makemigrations
python manage.py migrate
```

You now have:
- author always present
- ownership rules can be enforced reliably

---

## 12.11 Update ArticleForm Workflow to Set `author` Automatically

Users should **not** choose their author from a dropdown. That’s a security problem.

So we do:

- form does not include `author`
- view sets it using `commit=False`

### 12.11.1 Remove author from the form (if it appears)
In `articles/forms.py`, ensure `author` is not in `fields`.

Your current `fields` list doesn’t include it—good.

### 12.11.2 Update create view to set author
In `articles/views.py`:

```python
from django.contrib.auth.decorators import login_required
from django.views.decorators.http import require_http_methods

from .forms import ArticleForm

@login_required
@require_http_methods(["GET", "POST"])
def article_create(request):
    if request.method == "POST":
        form = ArticleForm(request.POST)
        if form.is_valid():
            article = form.save(commit=False)
            article.author = request.user
            article.save()
            form.save_m2m()
            ...
```

#### Why `save_m2m()` is required
Because when you use `commit=False`:
- Django hasn’t saved the object yet (no primary key)
- Many-to-many relations require a saved instance
So the correct sequence is:

1. `article = form.save(commit=False)`
2. set fields (author)
3. `article.save()`
4. `form.save_m2m()`

This pattern is essential for real apps.

---

## 12.12 Enforce Ownership on Edit (Object-Level Authorization)

Now implement:

- author can edit
- staff can edit
- everyone else gets 403

### 12.12.1 Add an authorization check in `article_edit`
In `articles/views.py`:

```python
from django.core.exceptions import PermissionDenied
from django.shortcuts import get_object_or_404

@login_required
@require_http_methods(["GET", "POST"])
def article_edit(request, slug: str):
    article = get_object_or_404(Article, slug=slug)

    is_owner = article.author_id == request.user.id
    is_staff = request.user.is_staff

    if not (is_owner or is_staff):
        raise PermissionDenied("You cannot edit this article.")
    ...
```

#### Why raise `PermissionDenied`
It triggers Django’s 403 handling.
This is the correct status for “resource exists, but you are not allowed.”

### 12.12.2 Add the same rule for delete (if you implemented delete exercise)
Any destructive action must be protected similarly.

---

## 12.13 Permissions and Groups (Model-Level Authorization)

Django automatically creates model permissions:

- `articles.add_article`
- `articles.change_article`
- `articles.delete_article`
- `articles.view_article`

### 12.13.1 When to use permissions vs ownership checks
- Ownership checks are object-level (this specific article).
- Permissions are role-level (this kind of action in general).

Professional pattern:
- Use permissions for broad capabilities (editors can edit articles).
- Use ownership for “only modify your own things.”

Often you combine them:
- allow edit if user has `change_article` OR user is owner.

### 12.13.2 Enforce permission in a view (optional pattern)
```python
from django.contrib.auth.decorators import permission_required

@permission_required("articles.add_article", raise_exception=True)
def article_create(...):
    ...
```

This is useful if:
- only certain roles can create
- you don’t want “every logged-in user can post”

If you want “any authenticated user can create,” then login_required is enough.

### 12.13.3 Create a Group “Editors” and assign permissions (admin)
In Django admin:
1. Go to **Groups**
2. Create group: `Editors`
3. Add permissions:
   - `articles | article | Can add article`
   - `articles | article | Can change article`
   - `articles | article | Can view article`
4. Create a user with `is_staff=True`
5. Add them to Editors group

Now they can manage articles (including admin and potentially views if you check
permissions there).

---

## 12.14 Password Reset Flow (Dev-Friendly Setup)

Password reset requires sending email. In development, you usually don’t want to
configure a real email provider.

### 12.14.1 Use console email backend (dev only)
In `config/settings.py`:

```python
EMAIL_BACKEND = "django.core.mail.backends.console.EmailBackend"
```

Now password reset emails print to the terminal.

### 12.14.2 Add minimal password reset templates
Django expects templates such as:
- `registration/password_reset_form.html`
- `registration/password_reset_done.html`
- `registration/password_reset_confirm.html`
- `registration/password_reset_complete.html`

At minimum, create `password_reset_form.html` so the page isn’t ugly.

Example `templates/registration/password_reset_form.html`:

```django
{% extends "base.html" %}

{% block title %}Reset password{% endblock %}

{% block content %}
  <h1>Reset password</h1>
  <form method="post">
    {% csrf_token %}
    {{ form.email.label_tag }}
    {{ form.email }}
    {% for err in form.email.errors %}<div class="error">{{ err }}</div>{% endfor %}
    <button type="submit">Send reset email</button>
  </form>
{% endblock %}
```

Then visit:
- `/accounts/password_reset/`
Submit your email. Check console output for the reset link.

---

## 12.15 Tests: Prove Auth and Authorization Rules Work

Create `articles/tests_auth.py`:

```python
from django.contrib.auth import get_user_model
from django.test import TestCase
from django.urls import reverse
from django.utils import timezone

from articles.models import Article


class ArticleAuthTests(TestCase):
    def setUp(self):
        User = get_user_model()
        self.u1 = User.objects.create_user(
            username="u1",
            password="pass12345",
        )
        self.u2 = User.objects.create_user(
            username="u2",
            password="pass12345",
        )

        self.article = Article.objects.create(
            title="Owned",
            slug="owned",
            body="Body",
            status=Article.Status.PUBLISHED,
            published_at=timezone.now(),
            author=self.u1,
        )

    def test_create_requires_login(self):
        url = reverse("articles:create")
        response = self.client.get(url)
        self.assertEqual(response.status_code, 302)
        self.assertIn("/accounts/login/", response["Location"])

    def test_owner_can_edit(self):
        self.client.login(username="u1", password="pass12345")
        url = reverse("articles:edit", kwargs={"slug": "owned"})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 200)

    def test_non_owner_cannot_edit(self):
        self.client.login(username="u2", password="pass12345")
        url = reverse("articles:edit", kwargs={"slug": "owned"})
        response = self.client.get(url)
        self.assertEqual(response.status_code, 403)
```

### Explanation of what these tests prove
- Anonymous users are not allowed to access create.
- Owners can edit.
- Non-owners receive 403 (not “404”; you are choosing to reveal existence).

**Security note:** Some apps prefer returning 404 to unauthorized users to avoid
information leakage (“resource exists”). Either is valid depending on threat model.
Django’s default is to use 403 for permission denied.

---

## 12.16 Common Auth Mistakes (and Fixes)

### Mistake A: “Login works but `request.user` is always AnonymousUser”
Likely cause:
- `AuthenticationMiddleware` missing
- `SessionMiddleware` missing
- middleware order wrong

Fix:
- ensure both exist and SessionMiddleware comes before AuthenticationMiddleware

### Mistake B: CSRF failure on login POST
Cause:
- missing `{% csrf_token %}` in login template (if you override it)
Fix:
- include `{% csrf_token %}` inside the login form

### Mistake C: “Anyone can edit by guessing the URL”
Cause:
- authentication check exists, but authorization check doesn’t
Fix:
- enforce object-level permission checks in edit/delete views

### Mistake D: “Users can set author via form”
Cause:
- author included in ModelForm fields
Fix:
- remove author from form
- set author in view using `commit=False`

---

## 12.17 Exercises (Do These Before Proceeding)

1. Add delete view with ownership protection:
   - only author or staff can delete
   - GET shows confirm page
   - POST deletes + redirects
   - write tests for owner vs non-owner

2. Add “My Articles” page:
   - requires login
   - lists only `Article.objects.filter(author=request.user)`
   - show it in nav only if logged in

3. Add permission-based creation:
   - require `articles.add_article` permission to access create
   - create an “Editors” group and assign the permission
   - test that a normal user gets 403, editor can create

4. Add password reset templates (done, confirm they render) and try the flow with
   console email backend.

---

## 12.18 Chapter Summary

- Authentication: identity (session-backed in standard Django).
- Authorization: permissions/ownership checks; must be enforced server-side.
- Django provides robust auth views; include `django.contrib.auth.urls`.
- Protect create/edit with `@login_required`.
- Implement object ownership using an `author` FK and enforce edit/delete rules.
- Use `commit=False` + `save_m2m()` when setting fields like author programmatically.
- Use groups/permissions for role-based access control.
- Write tests for redirects (anonymous) and 403 (unauthorized).

---

Next chapter: **Part II — Chapter 13: Middleware, Sessions, Messages, and Cookies**  
We’ll go deeper into the request pipeline, session storage backends, secure cookie
settings, messages internals, and write production-style custom middleware (request
IDs, structured logging hooks, security headers).